In [ ]:
# Title: Fig3----
# A régler: Si on prend 1978 pas tous les fichiers dans une année donc pb
# Normaliser? donnes entre 1980 et 210 
# Arthur BARREAU: 06 Février 2025

In [2]:
system("conda install -y r-sf=0.9_8")

In [8]:
system("conda install -y r-terra=1.2_10")

In [9]:
system("conda install -y r-dplyr")

In [10]:
system("conda install -y r-stringr")

In [11]:
system("conda install -y r-RColorBrewer")

In [ ]:
# 1 LOAD LIBRARIES ----
library(terra)
library(dplyr)
library(stringr)
library(RColorBrewer)
library(sf)
library(jsonlite)  # Ajouter cette ligne pour lire le JSON

# 2 VARIABLE ----
months_days <- list(
  Jan = 31, Feb = 28, Mar = 31, Apr = 30, May = 31, Jun = 30, 
  Jul = 31, Aug = 31, Sep = 30, Oct = 31, Nov = 30, Dec = 31
)

# 3 FUNCTION ----

# Récupérer la liste des fichiers dans galaxy_inputs/month ou galaxy_inputs/year
get_input_files <- function(input_path) {
  input_files <- list.files(input_path, full.names = TRUE)
  return(input_files)
}

# Validate the month and year
validate_month_year <- function(month, year) {
  if (!(month %in% names(months_days))) stop("Invalid month. Please enter a valid month.")
  if (is.na(year) || year < 1) stop("Invalid year. Please enter a valid year.")
  
  return(months_days[[month]])
}

# Create directories for storing TIFF and Shapefile data
create_directories <- function(year, month) {
  tif_path <- file.path("tif_folder", sprintf("%s_%s", year, month))
  zip_path <- "zip_folder"
  shp_path <- "shapefile_folder"
  dir.create(tif_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(zip_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(shp_path, showWarnings = FALSE, recursive = TRUE)
  
  return(list(tif_path = tif_path, zip_path = zip_path, shp_path = shp_path))
}

# Download daily TIFF files
download_tiff_files <- function(path, year, start_month, days_in_month) {
  month_index <- match(start_month, names(months_days))  
  month_folder <- sprintf("%02d_%s", month_index, start_month) 
  
  for (day in 1:days_in_month) {
    date_str <- sprintf("%s%02d%02d", year, month_index, day)  
    url <- paste0("https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/", year, "/", month_folder, "/S_", date_str, "_concentration_v3.0.tif")
    destfile <- file.path(path, sprintf("S_%s_concentration.tif", date_str))
    
    if (!file.exists(destfile)) {
      tryCatch({
        download.file(url, destfile)
      }, error = function(e) {
        message("Download failed for: ", url)
      })
    }
  }
}

# Compute the monthly mean raster
compute_monthly_mean <- function(path, start_year, start_month) {
  month_index <- match(start_month, names(months_days))
  dir.create("data", showWarnings = FALSE)
  tif_files <- list.files(path, pattern = "\\.tif$", full.names = TRUE)
  if (length(tif_files) > 0) {
    rasters <- rast(tif_files)
    rasters[rasters == 2550] <- NA  # Remplacer les valeurs 2550 par NA
    mean_raster <- mean(rasters, na.rm = TRUE)  
    output_raster <- sprintf("data/monthly_mean_%s_%02d.tif", start_year, month_index)
    writeRaster(mean_raster, output_raster, overwrite = TRUE)
    return(mean_raster)
  } else {
    message("No files downloaded. Check URLs and connection.")
    return(NULL)
  }
}


# Download and process shapefiles
download_shapefiles <- function(zip_path, shp_path, start_month) {
  month_index <- match(start_month, names(months_days))
  start_day <- sum(unlist(months_days)[1:(month_index - 1)]) + 1
  end_day <- start_day + months_days[[start_month]] - 1
  
  for (day in start_day:end_day) {
    days <- sprintf("%03d", day)
    url <- paste0("https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/shapefiles/dayofyear_median/median_extent_S_", days, "_1981-2010_polyline_v3.0.zip")
    destfile <- file.path(zip_path, sprintf("median_extent_S_%s.zip", days))
    
    tryCatch({
      download.file(url, destfile)
      unzip(destfile, exdir = shp_path)
    }, error = function(e) {
      message("Download failed for: ", url)
    })
  }
}

# Merge shapefiles and compute mean polygon
process_shapefiles <- function(shp_path) {
  shp_files <- list.files(shp_path, pattern = "\\.shp$", full.names = TRUE)
  polygons <- lapply(shp_files, st_read)
  merged_polygons <- do.call(st_union, polygons)
  mean_polygon <- st_intersection(merged_polygons)
  convex_hull <- st_convex_hull(mean_polygon)
  return(list(merged_polygons = merged_polygons, convex_hull = convex_hull))
}


# Create color palettes and breaks for visualization
create_colors_and_breaks <- function() {
  breaks <- c(0,1, seq(150, 1000, by = 50),1050, 2531, 2541)
  
  # Vérifier que les breaks sont uniques et triés
  breaks <- unique(sort(breaks))
  
  # Dégradé pour les glaciers (1 à 1000)
  colors_glacier <- rev(c(
    "#F7FCFF", "#E4F4FE", "#D0ECFE", "#BCE4FE", "#A8DCFD",
    "#94D5FD", "#7DCCFD", "#6AC4FC", "#57BCFC", "#45B4FC",
    "#31ABFC", "#23A3FC", "#1A9BFC", "#1994F9", "#178CF2",
    "#1684EB", "#137AE3", "#093C70" ))
  
  # Définition des couleurs
  colors <- c(
    rgb(9, 60, 112, maxColorValue = 255),
    colors_glacier,  
    rgb(255, 255, 255, maxColorValue = 255),  # Dégradé bleu glacier (1 à 1000)
    rgb(0, 0, 0, maxColorValue = 255),      # Ligne côtière noire (2530)
    rgb(119, 119, 119, maxColorValue = 255) # Terre grise (2540)
  )
  if (length(colors) != length(breaks) - 1) stop("Number of colors must match number of defined classes.")
  return(list(colors = colors, breaks = breaks))
}

# Visualization and saving results
save_plot <- function(raster, colors, breaks, polygons, year, month) {
  #dir.create("result", showWarnings = FALSE)
  png(filename = sprintf("outputs/Fig3_%s_%s.png", year, month), width = 800, height = 600)
  plot(raster, col = colors, breaks = breaks, legend = FALSE, axes = FALSE, box = FALSE)
  plot(st_geometry(polygons$merged_polygons), add = TRUE, pch = 21, col = 'red', cex = 1.5)
  dev.off()
}


# 4 MAIN ----

# Lire les paramètres depuis le fichier JSON
json_path <- "galaxy_inputs/galaxy_inputs.json"
json_data <- fromJSON(json_path)

start_month <- json_data$month
start_year <- json_data$year

days_in_month <- validate_month_year(start_month, start_year)
paths <- create_directories(start_year, start_month)

download_tiff_files(paths$tif_path, start_year, start_month, days_in_month)
mean_raster <- compute_monthly_mean(paths$tif_path, start_year, start_month)

download_shapefiles(paths$zip_path, paths$shp_path, start_month)
polygons <- process_shapefiles(paths$shp_path)

color_breaks <- create_colors_and_breaks()
save_plot(mean_raster, color_breaks$colors, color_breaks$breaks, polygons, start_year, start_month)
